In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
import torchvision
import numpy as np
import random
import time

In [2]:
import data_reader
SemData = data_reader.read_data_sets('data', padding=0, shuffle=True)

In [3]:
len(SemData.train.sentences)

98055

In [4]:
class Params():
    def __init__(self):
        self.n_inputs = 300
        self.n_hidden = 150
        self.n_class = 15
        self.batch_size = 128
        self.n_train = 2000
        self.n_display = 100
        self.test_size = 500

params = Params()

In [5]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.n_inputs = params.n_inputs
        self.n_hidden = params.n_hidden
        self.n_class = params.n_class
        self.batch_size = params.batch_size
        self.rnn = nn.RNN(self.n_inputs, self.n_hidden)
        self.lstm = nn.LSTM(self.n_inputs, self.n_hidden)
        self.fc = nn.Linear(self.n_hidden, self.n_class)
        self.h0 = torch.randn(1, 1, self.n_hidden).cuda()
        self.c0 = torch.randn(1, 1, self.n_hidden).cuda()
    
    def forward(self, sentences):
        for i in range(len(sentences)):
            sen_len = sentences[i].shape[0]
            sentence = sentences[i].reshape(1, sen_len, self.n_inputs)
            sentence = torch.tensor(sentence.transpose(1, 0, 2)).cuda()
            #_, h = self.rnn(sentence, self.h0)
            _, (h, _) = self.lstm(sentence, (self.h0, self.c0))
            h = torch.nn.functional.dropout(h, 0.5)
            out = self.fc(h[0])
            if i==0:
                output = out
            else:
                output = torch.cat([output, out])
        return output

In [6]:
model = RNN()
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.01)

In [7]:
def train():
    start = time.clock()
    for step in range(params.n_train):
        sentences, labels = SemData.train.next_batch(params.batch_size)
        labels = torch.tensor(labels).cuda()
        optimizer.zero_grad()
        logits = model(sentences)
        _, train_golden = torch.max(labels, 1)
        loss = criterion(logits, train_golden)
        loss.backward()
        optimizer.step()

        if step % params.n_display == 0:
            _, preds = torch.max(logits, 1)
            train_accuracy = torch.sum(preds == train_golden).item() / params.batch_size
            
            [test_sentences, test_labels] = SemData.test.next_batch(params.test_size)
            test_labels = torch.tensor(test_labels).cuda()
            
            test_logits = model(test_sentences)
            _, test_preds = torch.max(test_logits, 1)
            _, test_golden = torch.max(test_labels, 1)
            test_loss = criterion(test_logits, test_golden)
            test_accuracy = torch.sum(test_preds == test_golden).item() / params.test_size
            print("<step: %d>" % (step))
            print("train_accuracy: %.4g %% local_loss: %.8g" % (train_accuracy*100, loss.item()))
            print("test_accuracy: %.4g %% total_loss: %.8g" % (test_accuracy*100, test_loss.item()))
    print("------------------------------------")
    print("training time: ", time.clock()-start, " s")

    for _ in range(len(SemData.test.labels)//params.test_size):
        [test_sentences, test_labels] = SemData.test.next_batch(params.test_size)
        test_labels = torch.tensor(test_labels).cuda()
        test_logits = model(test_sentences)
        _, test_preds = torch.max(test_logits, 1)
        _, test_golden = torch.max(test_labels, 1)
        total_loss = 0
        total_accuracy = 0
        if total_loss == 0:
            total_loss = criterion(test_logits, test_golden).item()
        else:
            total_loss = total_loss + criterion(test_logits, test_golden).item()
        total_accuracy = total_accuracy + torch.sum(test_preds == test_golden).item() / params.test_size
        total_loss = total_loss / (len(SemData.test.labels)//params.test_size)
        total_accuracy = total_accuracy / (len(SemData.test.labels)//params.test_size)
    print("total_accuracy: %.4g %% total_loss: %.8g" % (total_accuracy*100, total_loss))

In [8]:
model = train()

<step: 0>
train_accuracy: 1.562 % local_loss: 2.791641
test_accuracy: 21.2 % total_loss: 2.6285262
<step: 100>
train_accuracy: 84.38 % local_loss: 0.67422211
test_accuracy: 78.4 % total_loss: 0.81282979
<step: 200>
train_accuracy: 76.56 % local_loss: 0.90772069
test_accuracy: 78.6 % total_loss: 0.819933
<step: 300>
train_accuracy: 83.59 % local_loss: 0.66814953
test_accuracy: 82.6 % total_loss: 0.7123028
<step: 400>
train_accuracy: 77.34 % local_loss: 0.89495814
test_accuracy: 80.4 % total_loss: 0.78801566
<step: 500>
train_accuracy: 78.91 % local_loss: 0.84024745
test_accuracy: 78.8 % total_loss: 0.85520715
<step: 600>
train_accuracy: 78.91 % local_loss: 0.83275414
test_accuracy: 77.6 % total_loss: 0.89844316
<step: 700>
train_accuracy: 82.81 % local_loss: 0.68820053
test_accuracy: 81.4 % total_loss: 0.72525257
<step: 800>
train_accuracy: 72.66 % local_loss: 0.97764891
test_accuracy: 79.6 % total_loss: 0.79711908
<step: 900>
train_accuracy: 82.03 % local_loss: 0.70341337
test_accuracy

UnboundLocalError: local variable 'total_accuracy' referenced before assignment

In [18]:
for _ in range(len(SemData.test.labels)//params.test_size):
    [test_sentences, test_labels] = SemData.test.next_batch(params.test_size)
    test_labels = torch.tensor(test_labels).cuda()
    test_logits = model(test_sentences)
    _, test_preds = torch.max(test_logits, 1)
    _, test_golden = torch.max(test_labels, 1)
    total_loss = 0
    total_accuracy = 0
    if total_loss == 0:
        total_loss = criterion(test_logits, test_golden).item()
    else:
        total_loss = total_loss + criterion(test_logits, test_golden).item()
    total_accuracy = total_accuracy + torch.sum(test_preds == test_golden).item() / params.test_size
    total_loss = total_loss / (len(SemData.test.labels)//params.test_size)
    total_accuracy = total_accuracy / (len(SemData.test.labels)//params.test_size)
print("total_accuracy: %.4g %% total_loss: %.8g" % (total_accuracy*100, total_loss))

total_accuracy: 3.752 % total_loss: 0.039092904
